## Photometry on MIRI images

In [ ]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
import warnings
import subprocess
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astropy.io import fits
from astropy.wcs import WCS, FITSFixedWarning
from astropy.table import Table
import astropy.units as u
from astropy.stats import SigmaClip

from photutils.aperture import EllipticalAperture, aperture_photometry
from photutils.background import Background2D, MedianBackground

warnings.simplefilter("ignore", category=FITSFixedWarning)


Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table.columns)

Now let's choose a galaxy, read its aperture data and overplot it!

In [ ]:
def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def adjust_aperture(galaxy_id, filter, survey, obs, output_folder, save_plot=False):
    
    # --- Load the FITS table ---
    #table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
    #aperture_table = Table.read(table_path)
    table_path =  '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'
    df = pd.read_csv(table_path)
    
    # --- Select the galaxy by ID ---
    #row = aperture_table[aperture_table['ID'] == galaxy_id][0]
    galaxy_id = int(galaxy_id)
    row = df[df['ID'] == galaxy_id].iloc[0]

    # --- Read in rotation angle of MIRI FITS file ---
    angle_file = '/home/bpc/University/master/Red_Cardinal/rotation_angles.json'
    with open(angle_file, "r") as f:
        angles = json.load(f)
    angle = angles[f"angle_{survey}{obs}"]
    
    # --- Read WCS from NIRCam image ---
    nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{galaxy_id}_F444W_cutout.fits"
    nircam_data, nircam_wcs = load_cutout(nircam_path)

    # --- Convert NIRCam pixel coordinates to sky ---
    sky_coord = nircam_wcs.pixel_to_world(row['Apr_Xcenter'], row['Apr_Ycenter'])

    # --- Open MIRI cutout image ---
    miri_path = f"/home/bpc/University/master/Red_Cardinal/cutouts/{galaxy_id}_{filter}_cutout_{survey}{obs}.fits"
    miri_data, miri_wcs = load_cutout(miri_path)

    # --- Convert sky coords to MIRI pixel coordinates ---
    miri_x, miri_y = miri_wcs.world_to_pixel(sky_coord)

    # --- Create elliptical region in MIRI pixel space ---
    nircam_scale = 0.03    # arcsec/pixel
    miri_scale = 0.11092  # arcsec per pixel
    
    # arcsec/pixel
    scale_factor = nircam_scale / miri_scale
    
    # --- Specify parameters for the ellips ---
    width = row['Apr_A'] * scale_factor
    height = row['Apr_B'] * scale_factor
    theta = -row['Apr_Theta']
    theta_new = ((theta - angle) % 180) * u.deg
    
    # --- Create region file and check if folder exists ---
    os.makedirs(output_folder, exist_ok=True)
    reg_file = os.path.join(output_folder, f'regions/{galaxy_id}_{survey}{obs}_aperture.reg') 
    
    # --- Write to DS9-compatible region file ---
    with open(reg_file, "w") as fh:
        fh.write("# Region file format: DS9 version 4.1\n")
        fh.write("global color=red dashlist=8 3 width=2 font=\"helvetica 10 normal\" "
                "select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        fh.write("image\n")
        fh.write(f"ellipse({miri_x:.2f},{miri_y:.2f},{width:.2f},{height:.2f},{theta_new:.2f})\n")

    if save_plot:
        
        # --- Clean and prepare the MIRI data for plotting ---
        miri_clean = np.copy(miri_data)
        finite_vals = miri_clean[np.isfinite(miri_clean)].flatten()
        
        # Sort and get lowest 80% values
        sorted_vals = np.sort(finite_vals)
        cutoff_index = int(0.8 * len(sorted_vals))
        background_vals = sorted_vals[:cutoff_index]
        background_mean = np.mean(background_vals)

        # Replace NaNs or infs with background mean
        miri_clean[~np.isfinite(miri_clean)] = background_mean

        fig, ax = plt.subplots(figsize=(6,6))
        ax.imshow(miri_clean, origin='lower', cmap='gray', vmin=np.percentile(miri_clean, 5), vmax=np.percentile(miri_clean, 99))

        # Original ellipse (without rotation correction)
        ellipse_original = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta,  # Just the original θ!
            edgecolor='red',
            facecolor='none',
            lw=2,
            label='Original Ellipse'
        )
        ax.add_patch(ellipse_original)

        ellipse = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta_new.to_value(u.deg),
            edgecolor='blue',
            linestyle='--',  # maybe dashed to differentiate
            facecolor='none',
            lw=2,
            label='Rotated Ellipse'
        )
        ax.add_patch(ellipse)
        
        
        
        ax.set_title(f"Galaxy {galaxy_id} - {filter} ({survey}{obs})")
        ax.set_xlim(miri_x - 30, miri_x + 30)
        ax.set_ylim(miri_y - 30, miri_y + 30)
        ax.legend(loc='upper right')
        
        # Save figure
        png_path = os.path.join(output_folder, f'masks/{galaxy_id}_{survey}{obs}_aperture_overlay.png')
        plt.savefig(png_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
        
    # Collect modified aperture data
    aperture_info = {
    'Flux': np.nan,           # You don't have these yet
    'Flux_Err': np.nan,        
    'Image_Err': np.nan,       
    'Flux_BKG': np.nan,        
    'AB_Mag': np.nan,          
    'Flux_BKG_Err': np.nan,    
    'N_PIX': np.nan,           
    'Apr_A': width,            # Rescaled aperture
    'Apr_B': height,
    'Apr_Xcenter': miri_x,
    'Apr_Ycenter': miri_y,
    'Apr_Theta': theta_new.to_value(u.deg),
    'ID': galaxy_id
    }
    
    return aperture_info
    

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=False)
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

df_path = '/home/bpc/University/master/Red_Cardinal/photometry/adjusted_apertures.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '20397'
filter = 'F770W'
survey_obs = 'cweb2'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()

# Section to perform the actual photometry

Imports

In [ ]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
import warnings
import subprocess
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astropy.io import fits
from astropy.wcs import WCS, FITSFixedWarning
from astropy.table import Table
import astropy.units as u
from astropy.stats import SigmaClip

from photutils.aperture import EllipticalAperture, aperture_photometry
from photutils.background import Background2D, MedianBackground

warnings.simplefilter("ignore", category=FITSFixedWarning)


First, let's set some parameters straight:

pixscale_arcsec = 0.11092  # arcsec per pixel

pix_area_sr = 2.89208962133982e-13  # from MIRI header


Functions

In [ ]:
# --- Functions ---

def get_psf(filter_name):
    """Read MIRI PSF file"""
    psf_file = f'/home/bpc/University/master/Red_Cardinal/WebbPSF/PSF_MIRI_{filter_name}.fits'
    with fits.open(psf_file) as psf:
        return psf[0].data

def get_aperture_params(galaxy_id, aperture_table):
    """Retrieve aperture parameters from the CSV table."""
    df = pd.read_csv(aperture_table)
    row = df[df['ID'] == int(galaxy_id)].iloc[0]
    return row['Apr_Xcenter'], row['Apr_Ycenter'], row['Apr_A'], row['Apr_B'], row['Apr_Theta'] * u.deg

def calculate_aperture_correction(psf_data, aperture_params):
    """Calculate aperture correction for given PSF and aperture."""
    aperture = EllipticalAperture(
        positions=(psf_data.shape[1] / 2, psf_data.shape[0] / 2),
        a=aperture_params['a'],
        b=aperture_params['b'],
        theta=aperture_params['theta']
    )
    total_flux = np.sum(psf_data)
    phot_table = aperture_photometry(psf_data, aperture)
    flux_in_aperture = phot_table['aperture_sum'][0]
    return total_flux / flux_in_aperture

def get_flux_and_uncertainty(data_bkgsub, aperture, background_rms):
    """Calculate flux and uncertainty from aperture photometry."""
    phot_table = aperture_photometry(data_bkgsub, aperture, method='exact')
    flux = phot_table['aperture_sum'][0]
    n_pix = np.pi * aperture.a * aperture.b
    flux_uncertainty = np.sqrt(n_pix) * np.median(background_rms)
    return flux, flux_uncertainty, n_pix

def get_background(data):
    """Subtract background using a model."""
    bkg = Background2D(
        data,
        box_size=5,
        filter_size=3,
        sigma_clip=SigmaClip(sigma=3.0),
        bkg_estimator=MedianBackground(),
        exclude_percentile=1.0,  # a bit less aggressive
        coverage_mask=np.isnan(data)
    )
    return bkg.background, bkg.background_rms, bkg.background_mesh_masked

def save_masked_cutout(galaxy_id, bkg, aperture_params, filter, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    # Create a Primary HDU
    hdu = fits.PrimaryHDU(bkg)
    
    # Add aperture info to header
    hdr = hdu.header
    hdr['Apr_Xcenter'] = aperture_params['x_center']
    hdr['Apr_Ycenter'] = aperture_params['y_center']
    hdr['Apr_A'] = aperture_params['a']
    hdr['Apr_B'] = aperture_params['b']
    hdr['Apr_Theta'] = aperture_params['theta']
    
    output_path = os.path.join(output_folder, f"{galaxy_id}_{filter}_masked.fits")
    hdu.writeto(output_path, overwrite=True)

    #print(f"Saved masked cutout for {galaxy_id} at {output_path}")
    
# --- Main Loop ---

def perform_photometry(cutout_files, aperture_table, psf_data, output_folder):
    """Main loop for photometry analysis."""
    photometry_results = []
    
    for fits_path in cutout_files:
        
        fits_name = os.path.basename(fits_path)
        galaxy_id = fits_name.split('_')[0]
        filter = fits_name.split('_')[1]
        
        with fits.open(fits_path) as hdul:
            data = hdul[1].data
        
        # Background subtraction
        background, background_rms, background_mesh_masked = get_background(data)
        data_bkgsub = data - background
        flux_bkg = np.median(background)
        flux_bkg_err = np.median(background_rms)
        
        #print(np.nanmin(background), np.nanmax(background))
        
        if np.allclose(data, data_bkgsub, equal_nan=True):
            print("Warning: background subtraction does not do anything!")
        
        # Aperture parameters
        miri_x, miri_y, width, height, theta = get_aperture_params(galaxy_id, aperture_table)
        aperture = EllipticalAperture((miri_x, miri_y), a=width / 2, b=height / 2, theta=theta.to_value(u.rad))

        # Photometry
        flux, flux_uncertainty, n_pix = get_flux_and_uncertainty(data_bkgsub, aperture, background_rms)

        # Aperture correction
        aperture_params = {'x_center': miri_x, 'y_center': miri_y, 'a': width / 2, 'b': height / 2, 'theta': theta.to_value(u.rad)}
        correction_factor = calculate_aperture_correction(psf_data, aperture_params)

        # Apply aperture correction
        flux_corrected = flux * correction_factor
        flux_uncertainty_corrected = flux_uncertainty * correction_factor
        flux_bkg_corrected = flux_bkg * correction_factor
        flux_bkg_err_corrected = flux_bkg_err * correction_factor

        # AB magnitude
        if flux_corrected > 0:
            ab_mag = -2.5 * np.log10(flux_corrected) + 8.90
        else: ab_mag = np.nan
        
        # Append results
        photometry_results.append({
            'Flux': flux_corrected,
            'Flux_Err': flux_uncertainty_corrected,
            'Image_Err': np.median(background_rms),
            'Flux_BKG': flux_bkg_corrected,
            'AB_Mag': ab_mag,
            'Flux_BKG_Err': flux_bkg_err_corrected,
            'N_PIX': n_pix,
            'Apr_A': width / 2,
            'Apr_B': height / 2,
            'Apr_Xcenter': miri_x,
            'Apr_Ycenter': miri_y,
            'Apr_Theta': theta.to_value(u.deg),
            'ID': int(galaxy_id)
        })

        # Save masked FITS files
        bkg_dir = os.path.join(output_folder, 'mosaic_fits/')
        os.makedirs(bkg_dir, exist_ok=True)
        save_masked_cutout(galaxy_id, background_mesh_masked, aperture_params, filter, bkg_dir)
        
    # Save to output table (assuming it's a pandas DataFrame)
    os.makedirs(output_folder, exist_ok=True)
    output_table = os.path.join(output_folder, f'results/photometry_table_{filter}.csv')
    output_df = pd.DataFrame(photometry_results)
    output_df.to_csv(output_table, index=False)
    


Now here is where the magic happens!

I wanna call the photometry function with:

perform_photometry(galaxy_id, filter, aperture_table, psf_data, cutouts_folder, output_table)


In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts/"
output_folder = '/home/bpc/University/master/Red_Cardinal/photometry/'
aperture_table = '/home/bpc/University/master/Red_Cardinal/photometry/adjusted_apertures.csv'

f770w_files = glob.glob(os.path.join(cutouts_folder, '*F770W*.fits'))
f1800w_files = glob.glob(os.path.join(cutouts_folder, '*F1800W*.fits'))

psf_f770w = get_psf('F770W')
psf_f1800w = get_psf('F1800W')
f770w_files = [f770w_files[1]]
perform_photometry(f770w_files, aperture_table, psf_f770w, output_folder)
#perform_photometry(f1800w_files, aperture_table, psf_f1800w, output_folder)

Now let's try and plot some of the images!

In [ ]:
# Open the saved masked cutout
background_dir = '/home/bpc/University/master/Red_Cardinal/photometry/mosaic_fits/'

cutout_path = os.path.join(background_dir, '22199_F770W_masked.fits')
with fits.open(cutout_path) as hdul:
    bkg = hdul[0].data

plt.imshow(bkg, origin='lower', cmap='gray')
plt.colorbar()
plt.show()

Now let's plot the results!

In [ ]:
table_dir = '/home/bpc/University/master/Red_Cardinal/photometry/results/'

df_fw770 = pd.read_csv(os.path.join(table_dir, 'photometry_table_F770W.csv'))
df_fw1800 = pd.read_csv(os.path.join(table_dir, 'photometry_table_F1800W.csv'))

# Create lookup dictionaries
photometry_fw770 = {row['ID']: row for _, row in df_fw770.iterrows()}
photometry_fw1800 = {row['ID']: row for _, row in df_fw1800.iterrows()}

# Find all unique IDs
all_ids = set(photometry_fw770.keys()).union(photometry_fw1800.keys())

Now let's try and create those juicy looking mosaics with the cutouts

In [ ]:
def plot_mosaic(galaxy_id, plotting_data_fw770, plotting_data_fw1800, output_dir):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6)) if galaxy_id in plotting_data_fw770 and galaxy_id in plotting_data_fw1800 else plt.subplots(1, 1, figsize=(6, 6))
    
    if isinstance(axs, plt.Axes):  # Only one subplot
        axs = [axs]

    if galaxy_id in plotting_data_fw770:
        data = plotting_data_fw770[galaxy_id]['background']
        mask = plotting_data_fw770[galaxy_id]['source_mask']
        masked_data = np.ma.array(data, mask=mask)
        cmap = plt.cm.viridis.copy()
        cmap.set_bad('white')
        axs[0].imshow(masked_data, origin='lower', cmap=cmap)
        axs[0].set_title(f'FW770 - ID {galaxy_id}')
        axs[0].axis('off')

    if galaxy_id in plotting_data_fw1800 and len(axs) > 1:
        data = plotting_data_fw1800[galaxy_id]['background']
        mask = plotting_data_fw1800[galaxy_id]['source_mask']
        masked_data = np.ma.array(data, mask=mask)
        cmap = plt.cm.viridis.copy()
        cmap.set_bad('white')
        axs[1].imshow(masked_data, origin='lower', cmap=cmap)
        axs[1].set_title(f'FW1800 - ID {galaxy_id}')
        axs[1].axis('off')
        
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'galaxy_{galaxy_id}.png'), dpi=300)

In [ ]:
photometry_dir = '/home/bpc/University/master/Red_Cardinal/photometry/'

with open(os.path.join(photometry_dir, 'results/plotting_table_F770W.pkl'), 'rb') as f:
    plotting_data_fw770 = pickle.load(f)

with open(os.path.join(photometry_dir, 'results/plotting_table_F1800W.pkl'), 'rb') as f:
    plotting_data_fw1800 = pickle.load(f)

output_dir = os.path.join(photometry_dir, 'mosaic_plots/')
for galaxy_id in all_ids:
    plot_mosaic(galaxy_id, plotting_data_fw770, plotting_data_fw1800, output_dir)